In [1]:
import pandas as pd
import requests
import json

In [21]:
urls = {
    "commodities": "https://apps.fas.usda.gov/OpenData/api/esr/commodities",
    "units": "https://apps.fas.usda.gov/OpenData/api/esr/unitsOfMeasure",
    "regions": "https://apps.fas.usda.gov/OpenData/api/esr/regions",
    "countries": "https://apps.fas.usda.gov/OpenData/api/esr/countries"
       }

In [4]:
key = open("usda_api_key.txt", "r").readlines()[0]

headers = {"API_KEY": key,
          "Accept": "application/json"}

In [7]:
commodities = requests.get(commodities_url, headers=headers)
commodities = json.loads(commodities.text)

In [22]:
json_dict = dict()

for k, v in urls.items():
    json_dict[k] = json.loads(requests.get(v, headers=headers).text)

In [24]:
json_dict.keys()

dict_keys(['commodities', 'units', 'regions', 'countries'])

In [25]:
data_release_url = "https://apps.fas.usda.gov/OpenData/api/esr/datareleasedates"
dr = requests.get(data_release_url, headers=headers)

In [33]:
market_url = "https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/801/allCountries/marketYear/2023"

market = requests.get(market_url, headers=headers)

In [38]:
market_json = json.loads(market.content)

In [51]:
for k, v in json_dict.items():
    with open(f"{k}.json", "w") as w:
        w.write(json.dumps(v, indent=4))   